In [ ]:
# 0. Colab 파일 업로드 (한 번만 필요)
from google.colab import files
print("필요 파일 업로드: metro JSON(data-metro-station-1.0.0.json,data-metro-line-1.0.0.json), bus Excel(bus_stops_seoul.xlsx,bus_route.xlsx) 파일을 올려주세요.")
uploaded = files.upload()

필요 파일 업로드: metro JSON(data-metro-station-1.0.0.json,data-metro-line-1.0.0.json), bus Excel(bus_stops_seoul.xlsx,bus_route.xlsx) 파일을 올려주세요.


Saving weather_seasons_categorized.json to weather_seasons_categorized.json
Saving microdust_normalized.json to microdust_normalized.json
Saving metro_accident.json to metro_accident.json
Saving congestion_normalized.json to congestion_normalized.json
Saving metro_inside.json to metro_inside.json
Saving metro_station.json to metro_station.json
Saving metro_line.json to metro_line.json


In [ ]:
!pip install folium pandas networkx osmnx scipy geopy openpyxl cch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.5/112.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.5/100.5 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 338.0/338.0 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 108.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 116.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.3/491.3 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
# 1. 라이브러리 설치 및 import
import json
import os
import pandas as pd
import networkx as nx
import osmnx as ox
import numpy as np
from osmnx.distance import nearest_nodes
from scipy.spatial import cKDTree
from geopy.geocoders import Nominatim
from itertools import combinations, islice
from math import radians, sin, cos, sqrt, asin
from multiprocessing import Pool
import cch
from geopy.geocoders import Nominatim
from concurrent.futures import ProcessPoolExecutor
import heapq
from itertools import product
from collections import defaultdict

In [ ]:
# ───────────────────────────────────────────────────────────────────────────────
# 2. 실제 도로망 전체 로드 (network_type='drive')
#    - 서울 전역을 대상으로 OSMnx의 drive 네트워크를 가져옵니다.
#    - 로그가 너무 많으면 아래 주석을 해제하고 설정을 바꿀 수 있습니다.
# ox.config(log_console=True, use_cache=True)

road_graph = ox.graph_from_place("Seoul, South Korea", network_type="drive")

# 2.1. ROAD_COORD 캐시에 각 노드의 좌표(위도/경도) 저장
ROAD_COORD = {}
for node, data in road_graph.nodes(data=True):
    ROAD_COORD[node] = (data['y'], data['x'])

print("도로망 로드 완료")
print("  - 도로망 노드 수:", road_graph.number_of_nodes())
print("  - 도로망 엣지 수:", road_graph.number_of_edges())

도로망 로드 완료
  - 도로망 노드 수: 68325
  - 도로망 엣지 수: 190080


나중에 사용자 위치도 매핑 시켜야함. walk 그래프에.

In [ ]:
# Step1: 사용자 출발지, 도착지 입력 및 지오코딩

# 1.1 geolocator 초기화
geolocator = Nominatim(user_agent="route_planner")

# 1.2 지오코딩 함수 정의
def get_location(name):
    loc = geolocator.geocode(name, timeout=10)
    if not loc:
        loc = geolocator.geocode(f"{name}, Seoul", timeout=10)
    if not loc:
        raise ValueError(f"'{name}' 위치를 찾을 수 없습니다.")
    return loc.latitude, loc.longitude

# 1.3 사용자에게 출발지와 도착지 명칭 입력받기
start_name = input("출발지 (예: 고려대학교): ")
end_name   = input("도착지 (예: 한남힐스테이트): ")

# 1.4 입력값을 위경도로 변환
start_lat, start_lon = get_location(start_name)
end_lat,   end_lon   = get_location(end_name)

# 1.5 결과 확인용 출력
print(f"출발지 '{start_name}' -> 위도: {start_lat:.6f}, 경도: {start_lon:.6f}")
print(f"도착지 '{end_name}' -> 위도: {end_lat:.6f},   경도: {end_lon:.6f}")

출발지 (예: 고려대학교): 종로3가역
도착지 (예: 한남힐스테이트): 강남역
출발지 '종로3가역' -> 위도: 37.570440, 경도: 126.992324
도착지 '강남역' -> 위도: 37.499935,   경도: 127.026968


Haversine 직선 거리(미터): 10065.997667197036


In [ ]:
# 예시 좌표 (출발/도착 좌표)
start_lat, start_lon = 37.570440, 126.992324
end_lat,   end_lon   = 37.500211, 127.026735

u = ox.distance.nearest_nodes(road_graph, start_lon, start_lat)
v = ox.distance.nearest_nodes(road_graph, end_lon,   end_lat)
print("출발 노드 ID:", u, "도착 노드 ID:", v)


출발 노드 ID: 12234274109 도착 노드 ID: 3827606155


In [ ]:
# 노드가 제대로 선택되었다면 실제 최단 경로 길이를 구해 본다
try:
    length_m = nx.shortest_path_length(road_graph, u, v, weight='length')
    print("도로망 최단경로 길이(미터):", length_m)
except nx.NetworkXNoPath:
    print("두 노드가 같은 연결 성분에 속해 있지 않습니다. (No path)")


도로망 최단경로 길이(미터): 9247.906975417733


In [ ]:
# 2. 유틸리티 함수 정의
def haversine(lat1, lon1, lat2, lon2):
    """두 지점 간 대원거리 계산 (미터)"""
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat, dlon = lat2 - lat1, lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    return 6371000 * c

def road_distance(lat1, lon1, lat2, lon2):
    """도로망 최단 경로 길이(미터) 또는 haversine fallback"""
    try:
        u = nearest_nodes(road_graph, lon1, lat1)
        v = nearest_nodes(road_graph, lon2, lat2)
        return nx.shortest_path_length(road_graph, u, v, weight='length')
    except (nx.NetworkXNoPath, Exception):
        return haversine(lat1, lon1, lat2, lon2)

In [ ]:
dist = road_distance(start_lat, start_lon, end_lat, end_lon)
print("도로망 기준 거리(미터):", dist)

도로망 기준 거리(미터): 9247.906975417733


In [ ]:
# 지하철 관련 전역 변수 (Step3에서 채울 예정)
subway_info = {}    # { station_name: {'lat': float, 'lng': float, 'lines': set([...])}, ... }
subway_graph = {}   # { (station_name, line): [ (neighbor_station, neighbor_line, weight), ... ], ... }

# 글로벌 속도 및 패널티 설정
WALK_SPEED = 5.0              # 보행 속도 5 km/h
WALK_PENALTY = 3.0            # 환승 시 부과되는 보행+대기 패널티 3분


In [ ]:
# 1~9호선 및 기타 전철·경전철 노선별로 직접 지정
speed_by_line = {
    '1호선': 28.6, '2호선': 36.5, '3호선': 34.9, '4호선': 30.1,
    '5호선': 32.8, '6호선': 30.0, '7호선': 32.3, '8호선': 32.2,
}

In [ ]:
import re

In [ ]:
def build_full_metro_graph(
    station_json: str,
    line_json:    str,
    speed_by_line: dict
):
    """
    1~8호선 전체 구간을 포함해 그래프 생성.
    inside_json 필터는 적용하지 않음.
    """
    stations_all = json.load(open(station_json,'r',encoding='utf-8'))['DATA']
    lines_all    = json.load(open(line_json,   'r',encoding='utf-8'))['DATA']

    # station_map: 모든 역 코드를 포함
    station_map = {
        str(s['station_cd']): {
            'name': s.get('name', s.get('station_nm')),
            'lat':  float(s['lat']),
            'lng':  float(s['lng'])
        }
        for s in stations_all
    }

    # 4) KDTree 준비
    station_ids = sorted(station_map.keys())
    coords      = np.array([
        [station_map[cd]['lat'], station_map[cd]['lng']]
        for cd in station_ids
    ])
    station_tree = cKDTree(coords)

    # 5) 도로망 노드 매핑
    lats       = [station_map[cd]['lat'] for cd in station_ids]
    lons       = [station_map[cd]['lng'] for cd in station_ids]
    road_nodes = nearest_nodes(road_graph, lons, lats)
    station_to_road = dict(zip(station_ids, road_nodes))

    # 6) Metro Graph 초기화
    Gm = nx.Graph()

    # 7) 1~8호선만 필터
    allowed_lines = set(speed_by_line.keys())
    lines = [info for info in lines_all if info['line'] in allowed_lines]

    # 8) 호선별 segment 엣지 추가 (양쪽 역 이름 모두 allowed_names 여야 함)
    for info in lines:
        line = info['line']
        kmh  = speed_by_line[line]
        for seg in info.get('node', []):
            u_cd = seg['station'][0]['station_cd']
            v_cd = seg['station'][1]['station_cd']
            # station_map 에 없으면 inside_json 에 없는 역 → 스킵
            if u_cd not in station_map or v_cd not in station_map:
                continue

            na = station_map[u_cd]
            nb = station_map[v_cd]
            # station_map['1007']['name'] == '신도림'
            u_node = f"{line}_{na['name']}"
            v_node = f"{line}_{nb['name']}"

            # 노드 추가
            for cd, nd, nm in [(u_cd, u_node, na), (v_cd, v_node, nb)]:
                if not Gm.has_node(nd):
                    Gm.add_node(
                        nd,
                        station   = nd.split('_',1)[1],  # na['name']
                        line      = line,
                        lat       = nm['lat'],
                        lng       = nm['lng'],
                        station_cd= cd
                    )

            # 도로망 거리 또는 haversine
            try:
                dist_m = nx.shortest_path_length(
                    road_graph,
                    station_to_road[u_cd],
                    station_to_road[v_cd],
                    weight='length'
                )
            except:
                dist_m = haversine(na['lat'], na['lng'], nb['lat'], nb['lng'])

            t_min = dist_m / 1000 / kmh * 60
            Gm.add_edge(u_node, v_node, weight=t_min, mode='metro_edge')

    # 9) 환승 엣지 추가 (같은 base station 이름끼리)
    transfer_groups = defaultdict(list)
    for node, data in Gm.nodes(data=True):
        transfer_groups[data['station']].append(node)

    for nodes in transfer_groups.values():
        if len(nodes) < 2:
            continue
        for u, v in combinations(nodes, 2):
            du, dv = Gm.nodes[u], Gm.nodes[v]
            walk_m = haversine(du['lat'], du['lng'], dv['lat'], dv['lng'])
            wt     = walk_m / 1000 / WALK_SPEED * 60 + WALK_PENALTY
            Gm.add_edge(u, v, weight=wt, mode='transfer')
            Gm.add_edge(v, u, weight=wt, mode='transfer')

    return Gm, station_ids, station_tree

In [ ]:
import json
from scipy.spatial import cKDTree

# 1) 전체 그래프 생성 (inside 필터 없이)
metro_graph, all_station_ids, all_station_tree = build_full_metro_graph(
    "metro_station.json",
    "metro_line.json",
    speed_by_line
)

# 2) inside_json 로 서울 구간 station_cd 집합 추출
inside_data  = json.load(open("metro_inside.json","r",encoding="utf-8"))
inside_codes = { str(item['station_cd']) for item in inside_data }

# 3) station_ids 재정의: inside_codes 에 들어 있는 코드만
station_ids = [ cd for cd in all_station_ids if cd in inside_codes ]

# 4) station_cd → (lat, lng) 맵 생성
station_coord_map = {}
for node, data in metro_graph.nodes(data=True):
    cd = data.get("station_cd")
    # inside 코드를 대상으로, 처음 만난 위경도만 저장
    if cd in inside_codes and cd not in station_coord_map:
        station_coord_map[cd] = (data["lat"], data["lng"])

# 5) station_ids 순서대로 coords 리스트 생성
coords = [ station_coord_map[cd] for cd in station_ids ]

# 6) KDTree 재생성
station_tree = cKDTree(coords)


In [ ]:
import json

# 1) metro_inside.json 로드해서 코드→이름 맵 생성
inside_all = json.load(open("metro_inside.json", "r", encoding="utf-8"))
cd_to_name = {
    str(item["station_cd"]): item.get("station_name", item.get("name"))
    for item in inside_all
}

# 2) 그래프에 없던 코드들 계산 (이전 코드)
codes_in_graph = { data["station_cd"] for _, data in metro_graph.nodes(data=True) }
missing_codes  = set(station_ids) - codes_in_graph

# 3) 코드→이름 변환
missing_names = [ cd_to_name.get(cd, cd) for cd in missing_codes ]

print("inside_list에는 있지만 그래프에 없는 station_cd:", missing_codes)
print("inside_list에는 있지만 그래프에 없는 station_name:", missing_names)


inside_list에는 있지만 그래프에 없는 station_cd: set()
inside_list에는 있지만 그래프에 없는 station_name: []


In [ ]:
print("inside_list total:", len(station_ids))
print("graph에 남은 역 코드 수:", len(codes_in_graph))


inside_list total: 156
graph에 남은 역 코드 수: 414


In [ ]:
line_split_nodes = metro_graph.number_of_nodes()
unique_stations  = len({data['station'] for _, data in metro_graph.nodes(data=True)})
print(f"line-split nodes: {line_split_nodes}, unique stations: {unique_stations}")


line-split nodes: 414, unique stations: 369


In [ ]:
import pandas as pd

# --- 1) 노드 정보 추출 및 DataFrame 생성 ---
nodes_data = [
    {
        'node_key': node,
        'station': data['station'],
        'line': data['line'],
        'lat': data['lat'],
        'lng': data['lng']
    }
    for node, data in metro_graph.nodes(data=True)
]
nodes_df = pd.DataFrame(nodes_data)
print(f"총 노드 수: {len(nodes_df)}")
display(nodes_df.head(10))  # 상위 10개 샘플 출력

# --- 2) 엣지 정보 추출 및 DataFrame 생성 ---
edges_data = [
    {
        'from': u,
        'to': v,
        'mode': data.get('mode'),
        'weight_min': data.get('weight')
    }
    for u, v, data in metro_graph.edges(data=True)
]
edges_df = pd.DataFrame(edges_data)
print(f"총 엣지 수: {len(edges_df)}")
display(edges_df.head(10))  # 상위 10개 샘플 출력

# --- 3) 예: 특정 역의 연결 확인 ---
station_of_interest = '신당'  # 예시: '신림'
mask = nodes_df['station'] == station_of_interest
nodes_of_interest = nodes_df.loc[mask, 'node_key'].tolist()

print(f"\n'{station_of_interest}'역에 해당하는 노드들:", nodes_of_interest)
for nk in nodes_of_interest:
    nbrs = metro_graph[nk]
    print(f"  - {nk}와 연결된 노드:")
    for nbr, attr in nbrs.items():
        print(f"      • {nbr} (mode={attr.get('mode')}, weight={attr.get('weight'):.2f}분)")


총 노드 수: 414


,node_key,station,line,lat,lng
0,1호선_연천,연천,1호선,38.100730,127.073720
1,1호선_전곡,전곡,1호선,38.024580,127.071800
2,1호선_청산,청산,1호선,37.981720,127.069120
3,1호선_소요산,소요산,1호선,37.948100,127.061034
4,1호선_동두천,동두천,1호선,37.927878,127.054790
5,1호선_보산,보산,1호선,37.913702,127.057277
6,1호선_동두천중앙,동두천중앙,1호선,37.901885,127.056482
7,1호선_지행,지행,1호선,37.892334,127.055716
8,1호선_덕정,덕정,1호선,37.843188,127.061277
9,1호선_덕계,덕계,1호선,37.818486,127.056486


총 엣지 수: 455


,from,to,mode,weight_min
0,1호선_연천,1호선_전곡,metro_edge,0.0
1,1호선_전곡,1호선_청산,metro_edge,0.0
2,1호선_청산,1호선_소요산,metro_edge,0.0
3,1호선_소요산,1호선_동두천,metro_edge,0.0
4,1호선_동두천,1호선_보산,metro_edge,0.0
5,1호선_보산,1호선_동두천중앙,metro_edge,0.0
6,1호선_동두천중앙,1호선_지행,metro_edge,0.0
7,1호선_지행,1호선_덕정,metro_edge,0.0
8,1호선_덕정,1호선_덕계,metro_edge,0.0
9,1호선_덕계,1호선_양주,metro_edge,0.0



'신당'역에 해당하는 노드들: ['2호선_신당', '6호선_신당']
  - 2호선_신당와 연결된 노드:
      • 2호선_상왕십리 (mode=metro_edge, weight=1.47분)
      • 2호선_동대문역사문화공원 (mode=metro_edge, weight=1.87분)
      • 6호선_신당 (mode=transfer, weight=6.73분)
  - 6호선_신당와 연결된 노드:
      • 6호선_청구 (mode=metro_edge, weight=1.34분)
      • 6호선_동묘앞 (mode=metro_edge, weight=1.28분)
      • 2호선_신당 (mode=transfer, weight=6.73분)


In [ ]:
# --- 1) 반경 내 지하철역 찾기 함수 ---
def nearby_metro_nodes(lat, lon, radius_m=500, graph=None, station_tree=None, station_ids=None):
    """
    KDTree를 이용해 반경 내 station_cd 인덱스를 찾고,
    해당 station_cd에 속한 모든 metro_graph 노드를 보행 시간 기준으로 반환합니다.
    """
    # 반경(m) → 도(degree) 단위로 근사 변환 (1도 ≈ 111km)
    radius_deg = radius_m / 111000.0

    # KDTree로 가까운 station_cd 인덱스 검색
    idxs = station_tree.query_ball_point([lat, lon], r=radius_deg)

    result = []
    for idx in idxs:
        cd = station_ids[idx]
        # 해당 station_cd에 속한 모든 line-분리 노드 탐색
        for node, data in graph.nodes(data=True):
            if data.get('station_cd') == cd:
                # 도로 기반 보행 시간 계산
                t_min = walk_time_via_road(lat, lon, data['lat'], data['lng'])
                result.append((node, t_min))

    # 보행 시간 기준 오름차순 정렬
    result.sort(key=lambda x: x[1])
    return result


In [ ]:
# --- 2) 도로망 보행 시간 계산 (기존과 동일) ---
def walk_time_via_road(lat1, lon1, lat2, lon2):
    try:
        u = nearest_nodes(road_graph, lon1, lat1)
        v = nearest_nodes(road_graph, lon2, lat2)
        dist_m = nx.shortest_path_length(road_graph, u, v, weight='length')
    except (nx.NetworkXNoPath, Exception):
        dist_m = haversine(lat1, lon1, lat2, lon2)
    return (dist_m / 1000) / WALK_SPEED * 60

In [ ]:
# inside 대신 all_ 트리를 넘기면…
start_candidates = nearby_metro_nodes(
    start_lat, start_lon,
    radius_m=500,
    graph=metro_graph,
    station_tree=all_station_tree,
    station_ids=all_station_ids
)
end_candidates = nearby_metro_nodes(
    end_lat, end_lon,
    radius_m=500,
    graph=metro_graph,
    station_tree=all_station_tree,
    station_ids=all_station_ids
)

# 거리순 정렬
start_candidates.sort(key=lambda x: x[1])
end_candidates.sort(key=lambda x: x[1])

print("출발 후보:", start_candidates)
print("도착 후보:", end_candidates)


출발 후보: [('1호선_종로3가', np.float64(0.017749571362737106)), ('3호선_종로3가', np.float64(1.2952134738131797)), ('5호선_종로3가', np.float64(4.194358523874303)), ('3호선_을지로3가', np.float64(13.302344320797138)), ('2호선_을지로3가', np.float64(14.209931287710392))]
도착 후보: [('2호선_강남', np.float64(11.654652421789013))]


In [ ]:
def format_time_mmss(t_min):
    total_sec = int(round(t_min * 60))
    m, s = divmod(total_sec, 60)
    return f"{m}분 {s}초"

# 기존 보행 시간 리스트 생성
start_nodes_with_walk = [
    (node,
     walk_time_via_road(start_lat, start_lon,
                        metro_graph.nodes[node]['lat'],
                        metro_graph.nodes[node]['lng']))
    for node, _ in start_candidates
]

end_nodes_with_walk = [
    (node,
     walk_time_via_road(metro_graph.nodes[node]['lat'],
                        metro_graph.nodes[node]['lng'],
                        end_lat, end_lon))
    for node, _ in end_candidates
]

# 포맷해서 상위 5개 출력
print("출발 후보 보행시간 (상위 5):")
for node, t_min in start_nodes_with_walk[:5]:
    print(f"{node}: {format_time_mmss(t_min)}")

print("\n도착 후보 보행시간 (상위 5):")
for node, t_min in end_nodes_with_walk[:5]:
    print(f"{node}: {format_time_mmss(t_min)}")


출발 후보 보행시간 (상위 5):
1호선_종로3가: 0분 1초
3호선_종로3가: 1분 18초
5호선_종로3가: 4분 12초
3호선_을지로3가: 13분 18초
2호선_을지로3가: 14분 13초

도착 후보 보행시간 (상위 5):
2호선_강남: 3분 32초


In [ ]:
import heapq

In [ ]:
def find_path_time(src, dst, graph, max_transfers):
    """
    메트로 그래프(graph) 상에서 src 에서 dst 까지의 최소 지하철 이동 시간을 구합니다.
    - max_transfers: 최대 환승 허용 횟수
    - 같은 호선을 내렸다가 다시 타는(재탑승) 경우는 불허용

    매개변수:
      src, dst   : 출발/도착 노드 ID (예: '2호선_을지로3가')
      graph      : metro_graph (networkx.Graph)
      max_transfers : 최대 환승 횟수 (int)

    반환값:
      최소 이동 시간(분) 또는 경로가 없으면 float('inf')
    """
    # 시작 노드의 호선을 가져와 초기 상태에 포함
    init_line = graph.nodes[src]['line']
    # 우선순위 큐: (지금까지 누적 이동시간, 현재 노드, 환승 횟수, 현재 호선, 방문한 호선 집합)
    pq = [(0.0, src, 0, init_line, {init_line})]
    # 이미 방문해서 알고 있는 최단 시간 기록 (키: (노드, 환승횟수, 현호선))
    seen = {}

    while pq:
        time, node, transfers, cur_line, visited_lines = heapq.heappop(pq)

        # 1) 도착지에 도달하면 그때까지 누적된 time 반환
        if node == dst:
            return time

        # 2) 같은 (node, transfers, cur_line) 조합으로 더 짧은 경로를
        #    이미 찾았다면 이 경로는 불필요
        key = (node, transfers, cur_line)
        if seen.get(key, float('inf')) <= time:
            continue
        seen[key] = time

        # 3) 인접 노드(이웃)로 확장
        for nbr, attr in graph[node].items():
            travel = attr['weight']               # 엣지의 이동 시간(분)
            next_line = graph.nodes[nbr]['line']  # 이웃 노드의 호선

            # 4) 환승 계산: 호선이 바뀌면 +1, 아니면 그대로
            new_trans = transfers + (1 if next_line != cur_line else 0)
            if new_trans > max_transfers:
                # 최대 환승 횟수를 넘으면 스킵
                continue

            # 5) 같은 호선을 내렸다가 다시 타는 경우 금지
            if next_line != cur_line and next_line in visited_lines:
                continue

            # 6) 방문한 호선 집합 갱신
            new_visited = visited_lines | {next_line}
            # 7) 다음 상태 큐에 추가
            heapq.heappush(
                pq,
                (time + travel, nbr, new_trans, next_line, new_visited)
            )

    # 모든 경로 탐색 후에도 도착 못 하면 무한대
    return float('inf')

In [ ]:
def get_fastest_subway_routes(
    G, origin_candidates, dest_candidates,
    k=5, max_transfers=4, max_paths_per_pair=10
):
    results = []

    for origin_node, origin_walk in origin_candidates:
        for dest_node, dest_walk in dest_candidates:
            try:
                paths_gen = nx.shortest_simple_paths(G, origin_node, dest_node, weight='weight')
            except nx.NetworkXNoPath:
                continue

            for path in islice(paths_gen, max_paths_per_pair):
                # 1) 환승 횟수
                transfers = sum(
                    1 for u,v in zip(path, path[1:])
                    if G[u][v].get('mode')=='transfer'
                )
                if transfers > max_transfers:
                    continue

                # 2) station 중복 순환 여부 체크 (기존)
                station_seq = [G.nodes[n]['station'] for n in path]
                reduced_stations = [station_seq[0]] + [
                    station_seq[i]
                    for i in range(1, len(station_seq))
                    if station_seq[i] != station_seq[i-1]
                ]
                if len(reduced_stations) != len(set(reduced_stations)):
                    continue

                # 3) line 재탑승(순환) 금지 추가
                line_seq = [G.nodes[n]['line'] for n in path]
                reduced_lines = [line_seq[0]] + [
                    line_seq[i]
                    for i in range(1, len(line_seq))
                    if line_seq[i] != line_seq[i-1]
                ]
                # 이미 탔던 호선이 또 등장하면 skip
                if len(reduced_lines) != len(set(reduced_lines)):
                    continue

                # 4) 시간 계산
                subway_time = sum(G[u][v]['weight'] for u,v in zip(path, path[1:]))
                total_time = subway_time + origin_walk + dest_walk

                results.append({
                    'path': path,
                    'total_time_min': total_time,
                    'subway_time_min': subway_time,
                    'walk_time_min': origin_walk + dest_walk,
                    'transfers': transfers
                })

    # 5) 정렬: 환승 횟수 우선, 그 다음 시간
    results.sort(key=lambda x: (x['transfers'], x['total_time_min']))
    return results[:k]


In [ ]:
# 1) 경로 생성
routes = get_fastest_subway_routes(
    metro_graph,
    start_nodes_with_walk,
    end_nodes_with_walk,
    k=200,
    max_transfers=4,
    max_paths_per_pair=100
)

# 2) 전체 경로 중 최소 소요 시간 구하기
fastest_time = min(r['total_time_min'] for r in routes) if routes else float('inf')

# 3) “최단 시간 × 2” 이하 경로만 필터링
candidate_routes = [
    r for r in routes
    if r['total_time_min'] <= 2 * fastest_time
]

# 4) 총 소요 시간 기준 오름차순 정렬
candidate_routes.sort(key=lambda r: r['total_time_min'])

# 5) 필터된 경로 출력 (예: 상위 200개)
for i, r in enumerate(candidate_routes[:200], 1):
    print(
        f"{i}위 ({r['total_time_min']:.1f}분, 환승 {r['transfers']}회): {r['path']}"
    )


1위 (38.8분, 환승 1회): ['3호선_종로3가', '3호선_을지로3가', '3호선_충무로', '3호선_동대입구', '3호선_약수', '3호선_금호', '3호선_옥수', '3호선_압구정', '3호선_신사', '3호선_잠원', '3호선_고속터미널', '3호선_교대', '2호선_교대', '2호선_강남']
2위 (42.1분, 환승 2회): ['1호선_종로3가', '3호선_종로3가', '3호선_을지로3가', '3호선_충무로', '3호선_동대입구', '3호선_약수', '3호선_금호', '3호선_옥수', '3호선_압구정', '3호선_신사', '3호선_잠원', '3호선_고속터미널', '3호선_교대', '2호선_교대', '2호선_강남']
3위 (46.7분, 환승 2회): ['5호선_종로3가', '3호선_종로3가', '3호선_을지로3가', '3호선_충무로', '3호선_동대입구', '3호선_약수', '3호선_금호', '3호선_옥수', '3호선_압구정', '3호선_신사', '3호선_잠원', '3호선_고속터미널', '3호선_교대', '2호선_교대', '2호선_강남']
4위 (46.7분, 환승 3회): ['1호선_종로3가', '1호선_종로5가', '1호선_동대문', '1호선_동묘앞', '6호선_동묘앞', '6호선_신당', '6호선_청구', '6호선_약수', '3호선_약수', '3호선_금호', '3호선_옥수', '3호선_압구정', '3호선_신사', '3호선_잠원', '3호선_고속터미널', '3호선_교대', '2호선_교대', '2호선_강남']
5위 (48.4분, 환승 3회): ['5호선_종로3가', '5호선_을지로4가', '5호선_동대문역사문화공원', '5호선_청구', '6호선_청구', '6호선_약수', '3호선_약수', '3호선_금호', '3호선_옥수', '3호선_압구정', '3호선_신사', '3호선_잠원', '3호선_고속터미널', '3호선_교대', '2호선_교대', '2호선_강남']
6위 (48.7분, 환승 1회): ['3호선_을지로3가', '3호선_충무로', '3호선_동대입구

In [ ]:
def categorize_by_time(results, time_key='total_time_min', cat_key='time_cat'):
    """
    results: [{'total_time_min':…, …}, …]
    time_key: 기준이 되는 시간 필드
    cat_key: 분류 결과를 저장할 새 필드
    """
    times = [r[time_key] for r in results]
    # 33rd, 66th percentile 계산
    q33, q66 = np.percentile(times, [33, 66])
    for r in results:
        t = r[time_key]
        if t <= q33:
            r[cat_key] = 1
        elif t <= q66:
            r[cat_key] = 0
        else:
            r[cat_key] = -1
    return results

# 사용 예시
routes = get_fastest_subway_routes(metro_graph,start_nodes_with_walk, end_nodes_with_walk)
categorized = categorize_by_time(routes)

for r in categorized:
    print(f"{r['total_time_min']:.2f}분 → 카테고리 {r['time_cat']}")


60.21분 → 카테고리 -1
38.76분 → 카테고리 1
48.67분 → 카테고리 1
53.41분 → 카테고리 0
54.63분 → 카테고리 -1


In [ ]:
# 1) 경로 생성
routes = get_fastest_subway_routes(
    metro_graph,
    start_nodes_with_walk,
    end_nodes_with_walk,
    k=100,               # 출력할 최종 경로 개수
    max_transfers=4,
    max_paths_per_pair=100
)

# 2) 시간 기준 카테고리 부여
categorized = categorize_by_time(routes, time_key='total_time_min', cat_key='time_cat')

# 3) 카테고리(1→0→–1) 우선, 시간 오름차순으로 정렬
#    key=(-time_cat, total_time_min) 으로 하면 1이 가장 앞, -1이 가장 뒤로 정렬됩니다.
sorted_routes = sorted(
    categorized,
    key=lambda r: (-r['time_cat'], r['total_time_min'])
)

# 4) 상위 100개 출력
for i, r in enumerate(sorted_routes[:100], 1):
    print(
        f"{i}위  {r['time_cat']}, "
        f"{r['total_time_min']:.1f}분, 환승 {r['transfers']}회): "
        f"{r['path']}"
    )


1위  1, 38.8분, 환승 1회): ['3호선_종로3가', '3호선_을지로3가', '3호선_충무로', '3호선_동대입구', '3호선_약수', '3호선_금호', '3호선_옥수', '3호선_압구정', '3호선_신사', '3호선_잠원', '3호선_고속터미널', '3호선_교대', '2호선_교대', '2호선_강남']
2위  1, 42.1분, 환승 2회): ['1호선_종로3가', '3호선_종로3가', '3호선_을지로3가', '3호선_충무로', '3호선_동대입구', '3호선_약수', '3호선_금호', '3호선_옥수', '3호선_압구정', '3호선_신사', '3호선_잠원', '3호선_고속터미널', '3호선_교대', '2호선_교대', '2호선_강남']
3위  1, 46.7분, 환승 2회): ['5호선_종로3가', '3호선_종로3가', '3호선_을지로3가', '3호선_충무로', '3호선_동대입구', '3호선_약수', '3호선_금호', '3호선_옥수', '3호선_압구정', '3호선_신사', '3호선_잠원', '3호선_고속터미널', '3호선_교대', '2호선_교대', '2호선_강남']
4위  1, 46.7분, 환승 3회): ['1호선_종로3가', '1호선_종로5가', '1호선_동대문', '1호선_동묘앞', '6호선_동묘앞', '6호선_신당', '6호선_청구', '6호선_약수', '3호선_약수', '3호선_금호', '3호선_옥수', '3호선_압구정', '3호선_신사', '3호선_잠원', '3호선_고속터미널', '3호선_교대', '2호선_교대', '2호선_강남']
5위  1, 48.4분, 환승 3회): ['5호선_종로3가', '5호선_을지로4가', '5호선_동대문역사문화공원', '5호선_청구', '6호선_청구', '6호선_약수', '3호선_약수', '3호선_금호', '3호선_옥수', '3호선_압구정', '3호선_신사', '3호선_잠원', '3호선_고속터미널', '3호선_교대', '2호선_교대', '2호선_강남']
6위  1, 48.7분, 환승 1회): ['3호선_을지로3가', '3

########### 혼잡도 함수

In [ ]:
df1 = pd.read_json('congestion_normalized.json', encoding='utf-8')
# 1) '호선' 칼럼을 문자열로 바꿔서 '2호선' 형태로 맞추기
df1['호선'] = df1['호선'].astype(str) + '호선'

In [ ]:
df1

,연번,요일구분,호선,역번호,출발역,상하구분,5시30분,6시00분,6시30분,7시00분,...,20시00분,20시30분,21시00분,21시30분,22시00분,22시30분,23시00분,23시30분,00시00분,00시30분
0,1,평일,1호선,158,청량리,상선,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1.0
1,2,평일,1호선,157,제기동,상선,1,1,1,1,...,1,1,0,1,1,1,1,1,1,1.0
2,3,평일,1호선,156,신설동,상선,1,1,1,1,...,1,1,0,1,1,1,1,1,1,1.0
3,4,평일,1호선,159,동묘앞,상선,1,1,1,1,...,1,1,0,1,1,1,1,1,1,1.0
4,5,평일,1호선,155,동대문,상선,1,1,1,1,...,0,1,0,0,1,1,1,1,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1657,1658,일요일,8호선,2823,남한산성입구,하선,1,1,1,1,...,1,1,1,1,1,1,1,1,1,NaN
1658,1659,일요일,8호선,2824,단대오거리,하선,1,1,1,1,...,1,1,1,1,1,1,1,1,1,NaN
1659,1660,일요일,8호선,2825,신흥,하선,1,1,1,1,...,1,1,1,1,1,1,1,1,1,NaN
1660,1661,일요일,8호선,2826,수진,하선,1,1,1,1,...,1,1,1,1,1,1,1,1,1,NaN


In [ ]:
# —————————————————————————————
# 1) JSON 읽기 & '호선' 문자열 보정
# —————————————————————————————
df1 = pd.read_json('congestion_normalized.json', encoding='utf-8')
df1['호선'] = df1['호선'].astype(str) + '호선'

# —————————————————————————————
# 2) 시각 컬럼 추출 & melt
# —————————————————————————————
time_cols = [c for c in df1.columns if re.match(r'\d+시', c)]
df_melt = df1.melt(
    id_vars=['호선', '출발역'],
    value_vars=time_cols,
    var_name='time',
    value_name='congestion'
)

# —————————————————————————————
# 3) line·출발역·time별 평균 혼잡도 계산
# —————————————————————————————
result = (
    df_melt
    .groupby(['호선', '출발역', 'time'], as_index=False)['congestion']
    .mean()
    .sort_values(['호선', '출발역', 'time'])
)

# —————————————————————————————
# 4) 전역(전체) congestion 분포에서 33%, 66% 분위수 계산
# —————————————————————————————
q33, q66 = result['congestion'].quantile([0.33, 0.66])

# —————————————————————————————
# 5) 카테고리 부여: 하위33%→1, 중간33~66%→0, 나머지→-1
# —————————————————————————————
result['congestion_cat'] = result['congestion'].apply(
    lambda x: 1 if x <= q33 else (0 if x <= q66 else -1)
)

# —————————————————————————————
# 6) 확인
# —————————————————————————————
print(result.head(15))

     호선  출발역    time  congestion  congestion_cat
0   1호선  동대문  00시00분    1.000000              -1
1   1호선  동대문  00시30분    1.000000              -1
2   1호선  동대문  10시00분    0.666667               0
3   1호선  동대문  10시30분    0.833333               0
4   1호선  동대문  11시00분    0.833333               0
5   1호선  동대문  11시30분    0.833333               0
6   1호선  동대문  12시00분    0.666667               0
7   1호선  동대문  12시30분    0.333333               1
8   1호선  동대문  13시00분    0.333333               1
9   1호선  동대문  13시30분    0.333333               1
10  1호선  동대문  14시00분    0.333333               1
11  1호선  동대문  14시30분    0.333333               1
12  1호선  동대문  15시00분    0.166667               1
13  1호선  동대문  15시30분    0.166667               1
14  1호선  동대문  16시00분    0.500000               1


In [ ]:
# 3.5 리스트 결합 후 멀티인덱스로 설정
congestion_df = pd.concat(result).set_index(['호선', '출발역', 'time_window'])


TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"

In [ ]:
from datetime import datetime

In [ ]:
# 4. 현재 시각에 속하는 시간대 반환 함수 정의
def current_time_window(now: datetime = None) -> str:
    """
    now: 확인할 datetime 객체. None이면 현재 시각(datetime.now()) 사용.
    반환값: time_windows 키 중 하나 ('morning_peak', 'evening' 등)
    """
    if now is None:
        now = datetime.now()
    hm = now.strftime('%H%M')  # '0730' 형식의 문자열로 변환

    # 각 시간대 범위에 따라 해당 키를 반환
    if '0530' <= hm < '0600':
        return 'early_morning'
    if '0700' <= hm <= '0930':
        return 'morning_peak'
    if '1000' <= hm <= '1530':
        return 'midday'
    if '1600' <= hm <= '1830':
        return 'evening_peak'
    if '1900' <= hm <= '2300':
        return 'evening'
    # 심야 구간: 자정 넘어서도 late_night 유지
    if hm >= '2330' or hm <= '0030':
        return 'late_night'
    # 기본값: 평일 낮 시간대로 처리
    return 'midday'


In [ ]:
# 5. 특정 호선·역의 현재 혼잡도 반환 함수
def get_station_congestion(line: str, station: str, now: datetime = None) -> float:
    """
    line: '2호선' 등 호선 이름 문자열
    station: '교대' 등 역 이름 문자열
    now: 확인할 datetime 객체. None이면 현재 시각 사용.
    반환값: 해당 호선·역·시간대의 평균 혼잡도 (float)
    """
    window = current_time_window(now)
    try:
        # 멀티인덱스로 조회
        return congestion_df.loc[(line, station, window), 'congestion']
    except KeyError:
        # 데이터 누락 시 전체 평균값 반환
        return congestion_df['congestion'].mean()


In [ ]:
if __name__ == "__main__":
    print("현재 시간대:", current_time_window())
    print("2호선 교대역 혼잡도:", get_station_congestion('2호선', '교대'))

현재 시간대: midday
2호선 교대역 혼잡도: 43.80833333333334


In [ ]:
def evaluate_congestion_for_candidates(
    G: nx.Graph,
    candidate_routes: list[dict],
    k: int = 10
) -> list[dict]:
    """
    candidate_routes: get_fastest_subway_routes → 필터링 후 얻은 리스트.
                      각 원소는 최소한 'path'와 'transfers'를 포함해야 합니다.
    반환: 각 경로에 'avg_congestion'을 추가하고, 낮은 것부터 상위 k개 반환
    """
    results = []
    for r in candidate_routes:
        path = r['path']
        # 1) 경로상의 모든 역에 대한 혼잡도 합산
        total_cong = 0.0
        for node in path:
            line    = G.nodes[node]['line']
            station = G.nodes[node]['station']
            total_cong += get_station_congestion(line, station)
        # 2) 평균 혼잡도
        avg_cong = total_cong / len(path)
        # 3) 원본 정보에 합치기
        entry = {
            **r,
            'avg_congestion': avg_cong
        }
        results.append(entry)

    # 4) 혼잡도 오름차순 정렬
    results.sort(key=lambda x: x['avg_congestion'])
    return results[:k]


In [ ]:
# --- 2) 필터된 후보에 대해 혼잡도 평가 ---
top_congestions = evaluate_congestion_for_candidates(
    metro_graph,
    candidate_routes,
    k=100
)

# --- 3) 결과 출력 예시 ---
for i, r in enumerate(top_congestions, 1):
    print(
        f"{i}위 (평균 혼잡도 {r['avg_congestion']:.3f}, "
        f"{r['total_time_min']:.1f}분, 환승 {r['transfers']}회): {r['path']}"
    )


1위 (평균 혼잡도 23.709, 55.1분, 환승 1회): ['1호선_종로3가', '1호선_종로5가', '1호선_동대문', '1호선_동묘앞', '1호선_신설동', '2호선_신설동', '2호선_용두', '2호선_신답', '2호선_용답', '2호선_성수', '2호선_건대입구', '2호선_구의', '2호선_강변', '2호선_잠실나루', '2호선_잠실', '2호선_잠실새내', '2호선_종합운동장', '2호선_삼성', '2호선_선릉', '2호선_역삼', '2호선_강남']
2위 (평균 혼잡도 23.825, 65.5분, 환승 2회): ['5호선_종로3가', '1호선_종로3가', '1호선_종로5가', '1호선_동대문', '1호선_동묘앞', '1호선_신설동', '2호선_신설동', '2호선_용두', '2호선_신답', '2호선_용답', '2호선_성수', '2호선_건대입구', '2호선_구의', '2호선_강변', '2호선_잠실나루', '2호선_잠실', '2호선_잠실새내', '2호선_종합운동장', '2호선_삼성', '2호선_선릉', '2호선_역삼', '2호선_강남']
3위 (평균 혼잡도 24.251, 61.0분, 환승 2회): ['3호선_종로3가', '1호선_종로3가', '1호선_종로5가', '1호선_동대문', '1호선_동묘앞', '1호선_신설동', '2호선_신설동', '2호선_용두', '2호선_신답', '2호선_용답', '2호선_성수', '2호선_건대입구', '2호선_구의', '2호선_강변', '2호선_잠실나루', '2호선_잠실', '2호선_잠실새내', '2호선_종합운동장', '2호선_삼성', '2호선_선릉', '2호선_역삼', '2호선_강남']
4위 (평균 혼잡도 24.339, 67.6분, 환승 3회): ['3호선_종로3가', '5호선_종로3가', '1호선_종로3가', '1호선_종로5가', '1호선_동대문', '1호선_동묘앞', '1호선_신설동', '2호선_신설동', '2호선_용두', '2호선_신답', '2호선_용답', '2호선_성수', '2호선_건대입구', '2호선_구의', '

############ 정시성

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from itertools import islice
# 1. metro_inside_info.json 로드
with open('metro_inside.json', 'r', encoding='utf-8') as f:
    info = json.load(f)
df = pd.DataFrame(info)

In [ ]:
# 2. lines_by_name 타입 보정
def parse_lines(x):
    if isinstance(x, list):
        return x
    if isinstance(x, str):
        return x.split(',')
    # 결측치 등
    return []

df['lines_by_name'] = df['lines_by_name'].apply(parse_lines)

# 3. station_name에서 "(숫자)" 제거
df['station_clean'] = df['station_name'].str.replace(r'\(.*\)$', '', regex=True)

# 4. score 계산
df['log_accident'] = np.log1p(df['accident_count'])
df['delay_accident_score'] = df['avg_delay_minutes'] + df['log_accident']


In [ ]:
# 2. Robust Scaling 함수 (지연시간 전용)----> 나중에 보고 적용할듯?
def robust_scale(series: pd.Series) -> pd.Series:
    """
    median/IQR 기반 스케일
    - 중앙값(median)을 0으로, IQR을 1로 스케일링
    - 이상치에 강인
    """
    med = series.median()
    iqr = series.quantile(0.75) - series.quantile(0.25)
    return (series - med) / iqr

# 3. 로그 변환 함수 (사고횟수 전용)
def log_transform(series: pd.Series) -> pd.Series:
    """
    np.log1p(x) 적용
    - x=0 → 0
    - 큰 값은 완만하게 증가
    """
    return np.log1p(series)


In [ ]:
# 4. 스케일링/변환 적용
df['log_accident']   = log_transform(df['accident_count'])

# 5. 결합 점수 계산 (클수록 불리)
#    필요에 따라 log_accident에 가중치 부여 가능 (예: *0.5)
df['delay_accident_score'] = df['avg_delay_minutes'] + df['log_accident']

In [ ]:
df['log_accident']

,log_accident
0,3.912023
1,2.484907
2,2.484907
3,3.367296
4,2.564949
...,...
331,0.000000
332,1.386294
333,2.484907
334,1.098612


In [ ]:
# 5. punct_map 생성
punct_map = {}
for _, row in df.iterrows():
    score = row['delay_accident_score']
    for line in row['lines_by_name']:
        key = f"{line}_{row['station_clean']}"
        punct_map[key] = max(punct_map.get(key, -np.inf), score)

지금 문제는 이름을 명명할때 겹치는게 있음.

In [ ]:
punct_map

{'1호선_신도림': 13.995603929228146,
 '2호선_신도림': 13.995603929228146,
 '1호선_동묘앞': 12.114010185188,
 '6호선_동묘앞': 12.114010185188,
 '1호선_창동': 14.404345773286474,
 '4호선_창동': 14.404345773286474,
 '1호선_가산디지털단지': 12.151522323961537,
 '7호선_가산디지털단지': 12.151522323961537,
 '1호선_신길': 11.884441773788,
 '5호선_신길': 11.884441773788,
 '1호선_동대문': 14.56341046791616,
 '4호선_동대문': 14.56341046791616,
 '1호선_종로5가': 12.767213454598371,
 '1호선_종로3가': 13.061020208529648,
 '3호선_종로3가': 13.061020208529648,
 '5호선_종로3가': 13.061020208529648,
 '1호선_종각': 12.767213454598371,
 '1호선_시청': 13.341677461821481,
 '2호선_시청': 13.341677461821481,
 '1호선_서울': 14.750622010004307,
 '4호선_서울': 14.750622010004307,
 '1호선_청량리': 13.66515504780433,
 '1호선_석계': 11.575013684455314,
 '6호선_석계': 11.575013684455314,
 '1호선_도봉산': 12.476944724396166,
 '7호선_도봉산': 12.476944724396166,
 '1호선_제기동': 13.077368382902211,
 '1호선_온수': 11.984468239298371,
 '7호선_온수': 11.984468239298371,
 '1호선_광명': 10.3693181818,
 '1호선_신설동': 12.856169646039781,
 '2호선_신설동': 12.856169646039781

In [ ]:
# 4. station별 점수 조회 함수
def get_station_punctuality(node: str, G: nx.Graph) -> float:
    line    = G.nodes[node]['line']
    station = G.nodes[node]['station']
    key = f"{line}_{station}"
    return punct_map.get(key, 0.0)

def evaluate_punctuality_for_candidates(
    G: nx.Graph,
    candidate_routes: list[dict],
    k: int = 10
) -> list[dict]:
    """
    candidate_routes: get_fastest_subway_routes → 필터링 후 얻은 리스트.
                      각 원소는 최소한 'path', 'transfers', 'total_time_min'을 포함해야 합니다.
    반환: 각 경로에 'punctuality_score'을 추가하고, 높은 점수 순으로 상위 k개 반환
    """
    results = []
    for r in candidate_routes:
        path = r['path']
        # 1) 그룹1: 출발역 + 환승역
        group1 = [path[0]] + [
            v for u, v in zip(path, path[1:])
            if G[u][v].get('mode') == 'transfer'
        ]
        group1 = list(dict.fromkeys(group1))
        # 2) 그룹2: 나머지 역
        group2 = [n for n in path if n not in group1]

        # 3) 평균 점수 함수
        def avg_score(nodes):
            if not nodes:
                return 0.0
            return sum(get_station_punctuality(n, G) for n in nodes) / len(nodes)

        g1_avg = avg_score(group1)
        g2_avg = avg_score(group2)

        # 4) 가중 합산 (그룹1:99%, 그룹2:1%)
        punctuality_score = 0.99 * g1_avg + 0.01 * g2_avg

        # 5) 원본 정보에 합치기
        entry = {
            **r,
            'punctuality_score': punctuality_score
        }
        results.append(entry)

    # 6) 정시성 우선 정렬 (높은 점수 우선), 동률은 환승·시간순
    results.sort(key=lambda x: (-x['punctuality_score'], x['transfers'], x['total_time_min']))
    return results[:k]


In [ ]:
# 3) 필터된 후보에 대해서만 정시성 평가
top_punctual = evaluate_punctuality_for_candidates(
    metro_graph,
    candidate_routes,
    k=100
)

# 4) 결과 출력
for i, r in enumerate(top_punctual, 1):
    print(
        f"{i}위 (정시성 {r['punctuality_score']:.3f}, "
        f"{r['total_time_min']:.1f}분, 환승 {r['transfers']}회): {r['path']}"
    )

1위 (정시성 14.215, 51.3분, 환승 2회): ['1호선_종로3가', '1호선_종각', '1호선_시청', '1호선_서울', '4호선_서울', '4호선_숙대입구', '4호선_삼각지', '4호선_신용산', '4호선_이촌', '4호선_동작', '4호선_이수', '4호선_사당', '2호선_사당', '2호선_방배', '2호선_서초', '2호선_교대', '2호선_강남']
2위 (정시성 14.172, 65.0분, 환승 3회): ['1호선_종로3가', '1호선_종로5가', '1호선_동대문', '4호선_동대문', '4호선_동대문역사문화공원', '5호선_동대문역사문화공원', '2호선_동대문역사문화공원', '2호선_신당', '2호선_상왕십리', '2호선_왕십리', '2호선_한양대', '2호선_뚝섬', '2호선_성수', '2호선_건대입구', '2호선_구의', '2호선_강변', '2호선_잠실나루', '2호선_잠실', '2호선_잠실새내', '2호선_종합운동장', '2호선_삼성', '2호선_선릉', '2호선_역삼', '2호선_강남']
3위 (정시성 14.162, 61.5분, 환승 2회): ['5호선_종로3가', '5호선_을지로4가', '5호선_동대문역사문화공원', '4호선_동대문역사문화공원', '4호선_충무로', '4호선_명동', '4호선_회현', '4호선_서울', '4호선_숙대입구', '4호선_삼각지', '4호선_신용산', '4호선_이촌', '4호선_동작', '4호선_이수', '4호선_사당', '2호선_사당', '2호선_방배', '2호선_서초', '2호선_교대', '2호선_강남']
4위 (정시성 14.157, 57.6분, 환승 2회): ['1호선_종로3가', '1호선_종로5가', '1호선_동대문', '4호선_동대문', '4호선_동대문역사문화공원', '4호선_충무로', '4호선_명동', '4호선_회현', '4호선_서울', '4호선_숙대입구', '4호선_삼각지', '4호선_신용산', '4호선_이촌', '4호선_동작', '4호선_이수', '4호선_사당', '2호선_사당', '2호선

############# 쾌적성

In [ ]:
from geopy.distance import geodesic
from datetime import datetime

In [ ]:
# 1) 데이터 로드
df_md = pd.read_json('metro_microdust.json')      # 미세먼지 데이터
df_ws = pd.read_json('weather_seasons.json')     # 계절별 기온 데이터


In [ ]:
df_md

,역명,호선,월평균,월
0,청량리,1호선,43.1,1
1,제기동,1호선,66.5,1
2,신설동,1호선,64.7,1
3,동묘,1호선,70.7,1
4,동대문,1호선,84.7,1
...,...,...,...,...
27550,서울대벤처타운,신림선,15.7,12
27551,관악산(서울대),신림선,13.7,12
27552,수서,GTX-A,47.9,12
27553,성남,GTX-A,65.0,12


In [ ]:
df_ws

,observatory_id,region_id,nam,address,lng,lat,spr_temp,season,sum_temp,fal_temp,win_temp
0,509.0,1,관악,서울특별시 관악구 신림동 산56-1 (서울대학교),126.950219,37.452863,11.438756,spring,NaN,NaN,NaN
1,417.0,2,금천,서울특별시 금천구 독산동 1034 (독산초등학교),126.914121,37.460314,12.775027,spring,NaN,NaN,NaN
2,401.0,3,서초,서울특별시 서초구 서초동 1650 (서울교육대학교),127.017913,37.482814,13.478883,spring,NaN,NaN,NaN
3,423.0,4,구로,서울특별시 구로구 궁동 213-42 (수궁동사무소),126.831229,37.486113,12.217729,spring,NaN,NaN,NaN
4,410.0,5,기상청,서울특별시 동작구 신대방동 460-18 (기상청),126.920722,37.496314,12.915244,spring,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
103,NaN,23,북한산,서울특별시 종로구 구기동 산1 (승가사),126.954419,37.618295,NaN,winter,NaN,NaN,-2.605182
104,NaN,24,노원,서울특별시 노원구 공릉동 230-3 (육군사관학교),127.087304,37.622196,NaN,winter,NaN,NaN,-0.941102
105,NaN,25,강북,서울특별시 강북구 수유동 192-49 (강북구청 본관),126.999613,37.636093,NaN,winter,NaN,NaN,0.434751
106,NaN,26,도봉,서울특별시 도봉구 방학동 310 (신방학초등학교),127.033110,37.666091,NaN,winter,NaN,NaN,-0.652582


In [ ]:
# 2) 역 좌표 목록 생성 (metro_graph에서 추출)
stations = []
for node, data in metro_graph.nodes(data=True):
    # 지하철 노드만 (예: "1호선_신도림")
    if not re.match(r'^\d+호선_', node):
        continue
    line, station = node.split('_', 1)
    # 괄호 제거
    station_clean = re.sub(r'\(.*\)$', '', station)
    stations.append({
        'node':    node,
        'line':    line,
        'station': station_clean,
        'lat':     data.get('lat'),
        'lng':     data.get('lng')
    })
df_st = pd.DataFrame(stations).drop_duplicates(subset=['line','station']).reset_index(drop=True)


In [ ]:
df_st

,node,line,station,lat,lng
0,1호선_연천,1호선,연천,38.100730,127.073720
1,1호선_전곡,1호선,전곡,38.024580,127.071800
2,1호선_청산,1호선,청산,37.981720,127.069120
3,1호선_소요산,1호선,소요산,37.948100,127.061034
4,1호선_동두천,1호선,동두천,37.927878,127.054790
...,...,...,...,...,...
409,8호선_남한산성입구,8호선,남한산성입구,37.451535,127.159816
410,8호선_단대오거리,8호선,단대오거리,37.445210,127.156866
411,8호선_신흥,8호선,신흥,37.440918,127.147564
412,8호선_수진,8호선,수진,37.437428,127.140722


In [ ]:
# 3) df_md 정리: df_st에 없는 역명 드롭
df_md = df_md[df_md['역명'].isin(df_st['station'])]

# 4) 미세먼지 월평균 계산 및 정규화
df_md_avg = (
    df_md
    .groupby(['호선', '역명'], as_index=False)['월평균']
    .mean()
    .rename(columns={'호선':'line', '역명':'station', '월평균':'microdust'})
)
md_scaler = MinMaxScaler()
df_md_avg['microdust_norm'] = md_scaler.fit_transform(df_md_avg[['microdust']])


In [ ]:
df_md_avg

,line,station,microdust,microdust_norm
0,1호선,교대,24.286364,0.124088
1,1호선,대곡,21.645455,0.096198
2,1호선,동대문,77.709091,0.688268
3,1호선,부평,27.745455,0.160618
4,1호선,부평구청,34.636364,0.233391
...,...,...,...,...
327,일산선,화정,28.045455,0.163786
328,하남선,미사,21.209091,0.091590
329,하남선,하남검단산,17.590909,0.053379
330,하남선,하남시청,18.636364,0.064420


In [ ]:
# 4) 계절 판별 함수
def classify_season(dt: datetime) -> str:
    m = dt.month
    if 3 <= m <= 5:  return 'spring'
    if 6 <= m <= 8:  return 'summer'
    if 9 <= m <= 11: return 'fall'
    return 'winter'

current_season = classify_season(datetime.now())

# 5) 날씨: 관측소별 온도를 계절별로 선택하는 헬퍼
def get_station_temp(lat, lng, season):
    col = {'spring':'spr_temp','summer':'sum_temp','fall':'fal_temp','winter':'win_temp'}[season]
    df_s = (
        df_ws[df_ws['season']==season]
        .dropna(subset=[col])
        .assign(lat=lambda d: pd.to_numeric(d['lat'], errors='coerce'),
                lng=lambda d: pd.to_numeric(d['lng'], errors='coerce'))
    )
    df_s['dist'] = df_s.apply(lambda r: geodesic((lat,lng),(r['lat'],r['lng'])).meters, axis=1)
    nearest = df_s.loc[df_s['dist'].idxmin()]
    return nearest[col]



In [ ]:
current_season

'summer'

In [ ]:
# 6) df_st 에 온도 추가
df_st['temp'] = df_st.apply(lambda r: get_station_temp(r['lat'], r['lng'], current_season), axis=1)

# 7) df_st + df_md_avg 병합 → df_all
df_all = pd.merge(df_st, df_md_avg[['line','station','microdust']], on=['line','station'], how='left')
df_all['microdust'] = df_all['microdust'].fillna(df_md_avg['microdust'].mean())

# 8) 범주별 점수화 함수 정의
def score_microdust_cat(x):
    if x <= 30:      return 4
    elif x <= 80:    return 3
    elif x <= 150:   return 2
    else:            return 1

def score_temp_cat(x):
    if 10 <= x <= 24:                                  return 4
    elif (0 <= x < 10) or (24 < x <= 28):              return 3
    elif (-10 <= x < 0) or (28 < x <= 34):             return 2
    else:                                              return 1

df_all['md_cat']   = df_all['microdust'].apply(score_microdust_cat)
df_all['temp_cat'] = df_all['temp'].apply(score_temp_cat)

# 9) 1–4점 범주를 0–1 정규화
df_all['md_scaled']   = (df_all['md_cat']   - 1) / 3
df_all['temp_scaled'] = (df_all['temp_cat'] - 1) / 3


In [ ]:
df_all

,node,line,station,lat,lng,temp,microdust,md_cat,temp_cat,md_scaled,temp_scaled
0,1호선_연천,1호선,연천,38.100730,127.073720,24.251172,32.356146,3,3,0.666667,0.666667
1,1호선_전곡,1호선,전곡,38.024580,127.071800,24.251172,32.356146,3,3,0.666667,0.666667
2,1호선_청산,1호선,청산,37.981720,127.069120,24.251172,32.356146,3,3,0.666667,0.666667
3,1호선_소요산,1호선,소요산,37.948100,127.061034,24.251172,32.356146,3,3,0.666667,0.666667
4,1호선_동두천,1호선,동두천,37.927878,127.054790,24.251172,32.356146,3,3,0.666667,0.666667
...,...,...,...,...,...,...,...,...,...,...,...
409,8호선_남한산성입구,8호선,남한산성입구,37.451535,127.159816,25.278920,22.863636,4,3,1.000000,0.666667
410,8호선_단대오거리,8호선,단대오거리,37.445210,127.156866,25.278920,16.145455,4,3,1.000000,0.666667
411,8호선_신흥,8호선,신흥,37.440918,127.147564,25.278920,24.536364,4,3,1.000000,0.666667
412,8호선_수진,8호선,수진,37.437428,127.140722,25.278920,32.445455,3,3,0.666667,0.666667


In [ ]:
# 10) 명시적 가중치로 쾌적도 계산
w_md, w_temp = 0.574509, 0.425491
df_all['pleasantness'] = df_all['md_scaled'] * w_md + df_all['temp_scaled'] * w_temp

# 11) (node → 쾌적도) 맵 생성
pleasant_map = dict(zip(df_all['node'], df_all['pleasantness']))

# 12) 결과 확인
df_all[['node', 'station', 'microdust', 'md_cat', 'md_scaled', 'temp', 'temp_cat', 'temp_scaled', 'pleasantness']].tail(10)

,node,station,microdust,md_cat,md_scaled,temp,temp_cat,temp_scaled,pleasantness
404,8호선_문정,문정,28.881818,4,1.000000,25.27892,3,0.666667,0.858170
405,8호선_장지,장지,25.381818,4,1.000000,25.27892,3,0.666667,0.858170
406,8호선_복정,복정,61.300000,3,0.666667,25.27892,3,0.666667,0.666667
407,8호선_남위례,남위례,32.356146,3,0.666667,25.27892,3,0.666667,0.666667
408,8호선_산성,산성,23.309091,4,1.000000,25.27892,3,0.666667,0.858170
409,8호선_남한산성입구,남한산성입구,22.863636,4,1.000000,25.27892,3,0.666667,0.858170
410,8호선_단대오거리,단대오거리,16.145455,4,1.000000,25.27892,3,0.666667,0.858170
411,8호선_신흥,신흥,24.536364,4,1.000000,25.27892,3,0.666667,0.858170
412,8호선_수진,수진,32.445455,3,0.666667,25.27892,3,0.666667,0.666667
413,8호선_모란,모란,15.463636,4,1.000000,25.27892,3,0.666667,0.858170


In [ ]:
# ——————————————
# 1) 헬퍼 함수 모음 (기존 정의된 상태)
# ——————————————

def get_station_pleasantness(node: str) -> float:
    """node: '2호선_신당' 형태 → 낮을수록 쾌적"""
    return pleasant_map.get(node, 0.0)

def classify_season(dt: datetime) -> str:
    m = dt.month
    if 3 <= m <= 5:   return 'spring'
    if 6 <= m <= 8:   return 'summer'
    if 9 <= m <= 11:  return 'fall'
    return 'winter'


## 추가

In [ ]:
def evaluate_pleasantness_for_candidates(
    G: nx.Graph,
    candidate_routes: list[dict],
    k: int = 10
) -> list[dict]:
    """
    candidate_routes: get_fastest_subway_routes → 필터링 후 얻은 리스트.
                      각 원소는 최소한 'path', 'transfers', 'total_time_min'을 포함해야 합니다.
    반환: 각 경로에 'pleasantness_score_sum'을 추가하고, 낮은 점수(더 쾌적) 순으로 상위 k개 반환
    """
    results = []
    for r in candidate_routes:
        path = r['path']
        # 1) 쾌적도 점수 합산 (낮을수록 더 쾌적)
        pleasant_sum = sum(get_station_pleasantness(n) for n in path)
        # 2) 원본 정보에 합치기
        entry = {
            **r,
            'pleasantness_score_sum': pleasant_sum
        }
        results.append(entry)

    # 3) 정렬: pleasant_sum ↑(낮은 게 우선), transfers ↑, total_time ↑
    results.sort(key=lambda x: (
        x['pleasantness_score_sum'],
        x['transfers'],
        x['total_time_min']
    ))
    return results[:k]


날씨에 대한 고민.

In [ ]:
# 3) 필터된 후보에 대해 쾌적도 평가
top_pleasant = evaluate_pleasantness_for_candidates(
    metro_graph,
    candidate_routes,
    k=100
)

# 4) 결과 출력
for i, r in enumerate(top_pleasant, 1):
    print(
        f"{i}위 (쾌적도 합계 {r['pleasantness_score_sum']:.3f}, "
        f"{r['total_time_min']:.1f}분, 환승 {r['transfers']}회): {r['path']}"
    )

1위 (쾌적도 합계 9.808, 48.7분, 환승 1회): ['3호선_을지로3가', '3호선_충무로', '3호선_동대입구', '3호선_약수', '3호선_금호', '3호선_옥수', '3호선_압구정', '3호선_신사', '3호선_잠원', '3호선_고속터미널', '3호선_교대', '2호선_교대', '2호선_강남']
2위 (쾌적도 합계 10.617, 38.8분, 환승 1회): ['3호선_종로3가', '3호선_을지로3가', '3호선_충무로', '3호선_동대입구', '3호선_약수', '3호선_금호', '3호선_옥수', '3호선_압구정', '3호선_신사', '3호선_잠원', '3호선_고속터미널', '3호선_교대', '2호선_교대', '2호선_강남']
3위 (쾌적도 합계 11.425, 42.1분, 환승 2회): ['1호선_종로3가', '3호선_종로3가', '3호선_을지로3가', '3호선_충무로', '3호선_동대입구', '3호선_약수', '3호선_금호', '3호선_옥수', '3호선_압구정', '3호선_신사', '3호선_잠원', '3호선_고속터미널', '3호선_교대', '2호선_교대', '2호선_강남']
4위 (쾌적도 합계 11.425, 46.7분, 환승 2회): ['5호선_종로3가', '3호선_종로3가', '3호선_을지로3가', '3호선_충무로', '3호선_동대입구', '3호선_약수', '3호선_금호', '3호선_옥수', '3호선_압구정', '3호선_신사', '3호선_잠원', '3호선_고속터미널', '3호선_교대', '2호선_교대', '2호선_강남']
5위 (쾌적도 합계 11.950, 48.4분, 환승 3회): ['5호선_종로3가', '5호선_을지로4가', '5호선_동대문역사문화공원', '5호선_청구', '6호선_청구', '6호선_약수', '3호선_약수', '3호선_금호', '3호선_옥수', '3호선_압구정', '3호선_신사', '3호선_잠원', '3호선_고속터미널', '3호선_교대', '2호선_교대', '2호선_강남']
6위 (쾌적도 합계 12.234, 48.8분, 환승 3

####### 환승

In [ ]:
import json
import re
import pandas as pd
import numpy as np
import networkx as nx
from geopy.distance import geodesic
from itertools import islice

# —————————————————————————
# 1) 데이터 로드 & 매핑표 생성
# —————————————————————————

# 1.1) 환승보행시간 데이터 로드
df_tt = pd.read_json('metro_transfer_time .json')
df_tt = (
    df_tt
    .rename(columns={
        '호선':'line',
        '환승역명':'station',
        '환승노선':'transfer_line',
        '환승소요시간':'duration_str'
    })
    # '1호선' 형태로 맞추고 "MM:SS" → 분 단위 float
    .assign(
        line=lambda df: df['line'].astype(str)+'호선',
        duration_min=lambda df: df['duration_str']
            .str.split(':')
            .apply(lambda t: int(t[0]) + int(t[1])/60)
    )
)
transfer_time_map = {
    (row.line, row.transfer_line, row.station): row.duration_min
    for _, row in df_tt.iterrows()
}

# 1.2) 지하철 내부 정보 로드 (에스컬레이터 길이)
with open('metro_inside.json','r',encoding='utf-8') as f:
    df_ins = pd.DataFrame(json.load(f))
# 괄호 제거된 역명 컬럼 추가
df_ins['station_clean'] = df_ins['station_name'].str.replace(r'\(.*\)$','',regex=True)
es_map = {
    (row['lines'], row['station_clean']): row['escalator_optimal_length']
    for _, row in df_ins.iterrows()
}

# 1.2.3) 지면 고도 맵: (호선, 역명) → ground_elevation (m)
ge_map = {
    (row['lines'], row['station_clean']): row['ground_elevation']
    for _, row in df_ins.iterrows()
}

# 보행·에스컬레이터 속도 정의 (m/min)
WALK_SPEED = 4000 / 60   # 4km/h → 약 66.7 m/min
ES_SPEED   = 8000 / 60   # 8km/h → 약133.3 m/min


In [ ]:
df_tt

,연번,line,station,transfer_line,환승거리,duration_str,duration_min
0,1,1호선,서울역,4호선,159,02:13,2.216667
1,2,1호선,서울역,공항철도,309,04:18,4.300000
2,3,1호선,서울역,경의중앙선,164,02:17,2.283333
3,4,1호선,시청,2호선,101,01:24,1.400000
4,5,1호선,종로3가,3호선,118,01:38,1.633333
...,...,...,...,...,...,...,...
135,136,8호선,석촌,9호선,82,01:08,1.133333
136,137,8호선,잠실,2호선,190,02:38,2.633333
137,138,8호선,가락시장,3호선,35,00:29,0.483333
138,139,8호선,복정,수인분당선,16,00:13,0.216667


In [ ]:
def vertical_time(line: str, station: str) -> float:
    """
    플랫폼 ↔ 지상 수직 이동 시간(분):
      - 지면고도 전체(ground_elev) 중 escalator 구간만 에스컬레이터 속도 적용
      - 나머지(ground_elev - escalator_length)는 보행속도 적용
    """
    ge = ge_map.get((line, station), 0.0)    # 총 수직고도(m)
    es = es_map.get((line, station), 0.0)    # 에스컬레이터 길이(m)
    # 에스컬레이터 구간
    t_es   = es / ES_SPEED
    # 나머지 구간은 보행
    walk_dist = max(0.0, ge - es)
    t_walk = walk_dist / WALK_SPEED
    return t_walk + t_es

In [ ]:
def evaluate_transfer_convenience_for_candidates(
    G: nx.Graph,
    candidate_routes: list[dict],
    w_tr: float = 0.383027,
    w_wk: float = 0.616973,
    k: int = 10
) -> list[dict]:
    evals = []

    # 1) 후보 경로별 convenience_time 계산
    for r in candidate_routes:
        path       = r['path']
        transfers  = r['transfers']
        walk_total = r['walk_time_min']  # origin_walk + dest_walk

        # 플랫폼 수직 이동
        line0, st0 = G.nodes[path[0]]['line'],  G.nodes[path[0]]['station']
        lineN, stN = G.nodes[path[-1]]['line'], G.nodes[path[-1]]['station']
        t_down = vertical_time(line0, st0)
        t_up   = vertical_time(lineN, stN)

        # 환승 보행시간 합산
        t_transfer = 0.0
        for u, v in zip(path, path[1:]):
            if G[u][v].get('mode') == 'transfer':
                ln      = G.nodes[u]['line']
                st      = G.nodes[u]['station']
                tr_line = G.nodes[v]['line']
                t_transfer += transfer_time_map.get((ln, tr_line, st), 0.0)

        convenience_time = walk_total + t_down + t_transfer + t_up

        evals.append({
            **r,
            'convenience_time': convenience_time
        })

    if not evals:
        return []

    # 2) Min–Max 정규화를 위한 최소/최대 추출 (항상 숫자 타입)
    transfers_list = [e['transfers']       for e in evals]
    conv_list      = [e['convenience_time'] for e in evals]
    min_tr, max_tr = min(transfers_list), max(transfers_list)
    min_cv, max_cv = min(conv_list),      max(conv_list)

    # 3) 정규화 & 가중합
    for e in evals:
        e['transfers_norm'] = (
            (e['transfers'] - min_tr) / (max_tr - min_tr)
            if max_tr > min_tr else 0.0
        )
        e['convenience_norm'] = (
            (e['convenience_time'] - min_cv) / (max_cv - min_cv)
            if max_cv > min_cv else 0.0
        )
        e['transfer_convenience_score'] = (
            e['transfers_norm'] * w_tr +
            e['convenience_norm'] * w_wk
        )

    # 4) 정렬 및 상위 k개
    evals.sort(key=lambda x: x['transfer_convenience_score'])
    return evals[:k]


In [ ]:
# 2) 실제로 후보 경로에 대해 호출
top_transfer_convenience = evaluate_transfer_convenience_for_candidates(
    metro_graph,
    candidate_routes,
    w_tr=0.383027,
    w_wk=0.616973,
    k=100
)

# 3) 출력 예시
for i, r in enumerate(top_transfer_convenience, 1):
    print(
        f"{i}위 (환승편의점수 {r['transfer_convenience_score']:.3f}, "
        f"환승 {r['transfers']}회, "
        f"편의시간 {r['convenience_time']:.1f}분, "
        f"총시간 {r['total_time_min']:.1f}분): {r['path']}"
    )

1위 (환승편의점수 0.140, 환승 1회, 편의시간 8.1분, 총시간 59.1분): ['1호선_종로3가', '1호선_종각', '1호선_시청', '2호선_시청', '2호선_을지로입구', '2호선_을지로3가', '2호선_을지로4가', '2호선_동대문역사문화공원', '2호선_신당', '2호선_상왕십리', '2호선_왕십리', '2호선_한양대', '2호선_뚝섬', '2호선_성수', '2호선_건대입구', '2호선_구의', '2호선_강변', '2호선_잠실나루', '2호선_잠실', '2호선_잠실새내', '2호선_종합운동장', '2호선_삼성', '2호선_선릉', '2호선_역삼', '2호선_강남']
2위 (환승편의점수 0.140, 환승 1회, 편의시간 8.1분, 총시간 63.6분): ['1호선_종로3가', '1호선_종각', '1호선_시청', '2호선_시청', '2호선_충정로', '2호선_아현', '2호선_이대', '2호선_신촌', '2호선_홍대입구', '2호선_합정', '2호선_당산', '2호선_영등포구청', '2호선_문래', '2호선_신도림', '2호선_대림', '2호선_구로디지털단지', '2호선_신대방', '2호선_신림', '2호선_봉천', '2호선_서울대입구', '2호선_낙성대', '2호선_사당', '2호선_방배', '2호선_서초', '2호선_교대', '2호선_강남']
3위 (환승편의점수 0.167, 환승 1회, 편의시간 8.9분, 총시간 55.1분): ['1호선_종로3가', '1호선_종로5가', '1호선_동대문', '1호선_동묘앞', '1호선_신설동', '2호선_신설동', '2호선_용두', '2호선_신답', '2호선_용답', '2호선_성수', '2호선_건대입구', '2호선_구의', '2호선_강변', '2호선_잠실나루', '2호선_잠실', '2호선_잠실새내', '2호선_종합운동장', '2호선_삼성', '2호선_선릉', '2호선_역삼', '2호선_강남']
4위 (환승편의점수 0.171, 환승 1회, 편의시간 9.0분, 총시간 38.8분): ['3호선_종로3가', '3호선_

raw_time        = total_time_min
raw_punctuality = 0.99*avg(group1_punct) + 0.01*avg(group2_punct)
raw_congestion  = avg_station_congestion
raw_pleasant    = sum_station_pleasantness
raw_transfer    = convenience_time

sum_time        = Σ(raw_time over all candidates)
sum_punct       = Σ(raw_punctuality over all candidates)
… (동일하게 sum_cong, sum_ples, sum_trans)

time_n     = raw_time        / sum_time
punct_n    = raw_punctuality / sum_punct
cong_n     = raw_congestion  / sum_cong
ples_n     = raw_pleasant    / sum_ples
trans_n    = raw_transfer    / sum_trans

// 정시성만 (1 – punct_n) 으로 전환
cost_time       = time_n
cost_punct      = 1 – punct_n
cost_congestion = cong_n
cost_pleasant   = ples_n
cost_transfer   = trans_n

weighted_score =
    w_time       * cost_time
  + w_punctuality* cost_punct
  + w_congestion * cost_congestion
  + w_pleasant   * cost_pleasant
  + w_transfer   * cost_transfer


In [ ]:
# 4) 통합 평가 함수: 5개 지표를 “합계 기반 비율 정규화” 후 가중합
# ——————————————————————————————————————————————————————————————————————

def evaluate_weighted_for_candidates(
    metro_graph: nx.Graph,
    candidate_routes: list[dict],
    weights: dict,
    k: int = 10
) -> list[dict]:
    """
    5개 지표(raw) → 비율 정규화(by-sum) → 가중합 → 상위 k개 반환

    weights = {
      'time':        w_time,       # 소요시간 가중치
      'punctuality': w_punct,      # 정시성 가중치
      'congestion':  w_cong,       # 혼잡도 가중치
      'pleasant':    w_ples,       # 쾌적성 가중치
      'transfer':    w_trans,      # 환승편의 가중치
    }  (sum == 1)
    """
    evals = []
    # 1) 각 경로별 원시 지표 계산
    for r in candidate_routes:
        path = r['path']
        # 1-1) 총 소요시간 (지하철+도보)
        t = r['total_time_min']
        # 1-2) 정시성: 출발역+환승역(99%) vs 나머지(1%)
        grp1 = [path[0]] + [
            v for u,v in zip(path, path[1:])
            if metro_graph[u][v].get('mode')=='transfer'
        ]
        grp1 = list(dict.fromkeys(grp1))
        grp2 = [n for n in path if n not in grp1]
        def _avg_punct(nodes):
            return (sum(get_station_punctuality(n, metro_graph) for n in nodes)
                    / len(nodes)) if nodes else 0.0
        punct = 0.99*_avg_punct(grp1) + 0.01*_avg_punct(grp2)
        # 1-3) 평균 혼잡도 (낮을수록 좋음)
        cong = sum(
            get_station_congestion(
                metro_graph.nodes[n]['line'],
                metro_graph.nodes[n]['station']
            ) for n in path
        ) / len(path)
        # 1-4) 쾌적성 합산 (낮을수록 좋음)
        ples = sum(get_station_pleasantness(n) for n in path)
        # 1-5) 환승편의성: 도보 + 수직이동 + 환승보행
        walk = r['walk_time_min']
        ln0, st0 = metro_graph.nodes[path[0]]['line'], metro_graph.nodes[path[0]]['station']
        ln1, st1 = metro_graph.nodes[path[-1]]['line'], metro_graph.nodes[path[-1]]['station']
        t_down = vertical_time(ln0, st0)
        t_up   = vertical_time(ln1, st1)
        t_tr   = 0.0
        for u,v in zip(path, path[1:]):
            if metro_graph[u][v].get('mode')=='transfer':
                ln, st = metro_graph.nodes[u]['line'], metro_graph.nodes[u]['station']
                t_tr += transfer_time_map.get((ln, metro_graph.nodes[v]['line'], st), 0.0)
        trans = walk + t_down + t_tr + t_up

        evals.append({
            **r,
            'time_m':  t,
            'punct_m': punct,
            'cong_m':  cong,
            'ples_m':  ples,
            'trans_m': trans,
        })

    # 2) raw 지표 합계 계산 (정규화 분모)
    sums = {k: sum(e[f"{k}_m"] for e in evals) for k in ['time','punct','cong','ples','trans']}

    # 3) 비율 정규화(by-sum) + 가중합
    for e in evals:
        # 3-1) 지표별 비율
        e['time_n']    = e['time_m']    / sums['time']    if sums['time']    > 0 else 0.0
        e['punct_n']   = e['punct_m']   / sums['punct']   if sums['punct']   > 0 else 0.0
        e['cong_n']    = e['cong_m']    / sums['cong']    if sums['cong']    > 0 else 0.0
        e['ples_n']    = e['ples_m']    / sums['ples']    if sums['ples']    > 0 else 0.0
        e['trans_n']   = e['trans_m']   / sums['trans']   if sums['trans']   > 0 else 0.0

        # 3-2) 비용(cost) 항으로 그대로 변환
        # - 소요시간, 정시성, 혼잡도, 쾌적성, 환승편의성 모두 “값이 작을수록 좋은” 비용이므로
        #   normalized 값 그대로 사용합니다.
        e['weighted_score'] = (
            weights['time']        * e['time_n']   +
            weights['punctuality'] * e['punct_n']  +
            weights['congestion']  * e['cong_n']   +
            weights['pleasant']    * e['ples_n']   +
            weights['transfer']    * e['trans_n']
        )

    # 4) weighted_score 오름차순 정렬
    evals.sort(key=lambda x: x['weighted_score'])

    # 5) inverted min–max 정규화: 최저(cost 큰)→0, 최고(cost 작은)→1
    scores = [e['weighted_score'] for e in evals]
    min_s, max_s = min(scores), max(scores)
    for e in evals:
        if max_s > min_s:
            norm = (e['weighted_score'] - min_s) / (max_s - min_s)
            e['score_norm_inv'] = 1.0 - norm
        else:
            e['score_norm_inv'] = 1.0

    # 6) 상위 k개 반환 (score_norm_inv 포함)
    return evals[:k]


In [ ]:
# ——————————————————————————————————————————————————————————————————————
# 5) 5가지 우선순위별 2차 가중치 정의 (합 = 1)
# ——————————————————————————————————————————————————————————————————————
w_punct_primary = {
    'time':        0.123831,
    'punctuality': 1.000000,
    'transfer':    0.354921,
    'congestion':  0.216503,
    'pleasant':    0.304746,
}

w_time_primary = {
    'time':        1.000000,
    'punctuality': 0.240689,
    'transfer':    0.303901,
    'congestion':  0.240822,
    'pleasant':    0.214589,
}

w_transfer_primary = {
    'time':        0.170219,
    'punctuality': 0.311810,
    'transfer':    1.000000,
    'congestion':  0.230867,
    'pleasant':    0.287104,
}

w_congestion_primary = {
    'time':        0.154357,
    'punctuality': 0.289804,
    'transfer':    0.321203,
    'congestion':  1.000000,
    'pleasant':    0.234636,
}

w_pleasant_primary = {
    'time':        0.163446,
    'punctuality': 0.326817,
    'transfer':    0.314097,
    'congestion':  0.195641,
    'pleasant':    1.000000,
}


In [ ]:
# ——————————————————————————————————————————————————————————————————————
# 6) 후보 경로 생성 (공통)
# ——————————————————————————————————————————————————————————————————————
routes = get_fastest_subway_routes(
    metro_graph,
    start_nodes_with_walk,
    end_nodes_with_walk,
    k=200
)
fastest = routes[0]['total_time_min']
candidates = [r for r in routes if r['total_time_min'] <= 2 * fastest]

# ——————————————————————————————————————————————————————————————————————
# 7) 각 우선순위별 평가 실행 및 출력 (score_norm_inv 사용)
# ——————————————————————————————————————————————————————————————————————
priority_weights = {
    '정시성 우선':     w_punct_primary,
    '소요시간 우선':   w_time_primary,
    '환승편의 우선':   w_transfer_primary,
    '혼잡도 우선':     w_congestion_primary,
    '쾌적성 우선':     w_pleasant_primary,
}

for label, weights in priority_weights.items():
    print(f"\n=== {label} Top 5 경로 ===")
    top5 = evaluate_weighted_for_candidates(
        metro_graph,
        candidates,
        weights,
        k=5
    )
    for i, r in enumerate(top5, 1):
        path_str = " → ".join(r['path'])
        print(f"{i}위 (score_norm_inv={r['score_norm_inv']:.3f}): {path_str}")



=== 정시성 우선 Top 5 경로 ===
1위 (score_norm_inv=1.000): 3호선_종로3가 → 3호선_을지로3가 → 3호선_충무로 → 3호선_동대입구 → 3호선_약수 → 3호선_금호 → 3호선_옥수 → 3호선_압구정 → 3호선_신사 → 3호선_잠원 → 3호선_고속터미널 → 3호선_교대 → 2호선_교대 → 2호선_강남
2위 (score_norm_inv=0.943): 1호선_종로3가 → 3호선_종로3가 → 3호선_을지로3가 → 3호선_충무로 → 3호선_동대입구 → 3호선_약수 → 3호선_금호 → 3호선_옥수 → 3호선_압구정 → 3호선_신사 → 3호선_잠원 → 3호선_고속터미널 → 3호선_교대 → 2호선_교대 → 2호선_강남
3위 (score_norm_inv=0.921): 1호선_종로3가 → 1호선_종로5가 → 1호선_동대문 → 1호선_동묘앞 → 1호선_신설동 → 2호선_신설동 → 2호선_용두 → 2호선_신답 → 2호선_용답 → 2호선_성수 → 2호선_건대입구 → 2호선_구의 → 2호선_강변 → 2호선_잠실나루 → 2호선_잠실 → 2호선_잠실새내 → 2호선_종합운동장 → 2호선_삼성 → 2호선_선릉 → 2호선_역삼 → 2호선_강남
4위 (score_norm_inv=0.877): 1호선_종로3가 → 1호선_종로5가 → 1호선_동대문 → 1호선_동묘앞 → 6호선_동묘앞 → 6호선_신당 → 6호선_청구 → 6호선_약수 → 3호선_약수 → 3호선_금호 → 3호선_옥수 → 3호선_압구정 → 3호선_신사 → 3호선_잠원 → 3호선_고속터미널 → 3호선_교대 → 2호선_교대 → 2호선_강남
5위 (score_norm_inv=0.852): 1호선_종로3가 → 1호선_종각 → 1호선_시청 → 1호선_서울 → 4호선_서울 → 4호선_숙대입구 → 4호선_삼각지 → 4호선_신용산 → 4호선_이촌 → 4호선_동작 → 4호선_이수 → 4호선_사당 → 2호선_사당 → 2호선_방배 → 2호선_서초 → 2호선_교대 → 2호선_강남

=== 소요시간 우선 Top 5 경로 ===

In [ ]:
# ——————————————————————————————————————————————————————————————————————
# 가) 최종 상대중요도(합=1)
w_final = {
    'time':        0.136274,   # 소요시간
    'punctuality': 0.233372,   # 정시성
    'transfer':    0.274455,   # 환승편의
    'congestion':  0.155965,   # 혼잡도
    'pleasant':    0.199933,   # 쾌적성
}

# 나) 후보 경로 생성 (한 번만)
routes = get_fastest_subway_routes(
    metro_graph,
    start_nodes_with_walk,
    end_nodes_with_walk,
    k=200
)
fastest    = routes[0]['total_time_min']
candidates = [
    r for r in routes
    if r['total_time_min'] <= 2 * fastest
]

print("=== 최종 종합만족도 우선 Top 5 경로 ===")
top5_final = evaluate_weighted_for_candidates(
    metro_graph,
    candidates,
    w_final,
    k=5
)
for i, r in enumerate(top5_final, 1):
    path_str = " → ".join(r['path'])
    # weighted_score 대신 score_norm_inv 사용
    print(f"{i}위 (score_norm_inv={r['score_norm_inv']:.3f}): {path_str}")

=== 최종 종합만족도 우선 Top 5 경로 ===
1위 (score_norm_inv=1.000): 3호선_종로3가 → 3호선_을지로3가 → 3호선_충무로 → 3호선_동대입구 → 3호선_약수 → 3호선_금호 → 3호선_옥수 → 3호선_압구정 → 3호선_신사 → 3호선_잠원 → 3호선_고속터미널 → 3호선_교대 → 2호선_교대 → 2호선_강남
2위 (score_norm_inv=0.939): 1호선_종로3가 → 1호선_종각 → 1호선_시청 → 1호선_서울 → 4호선_서울 → 4호선_숙대입구 → 4호선_삼각지 → 4호선_신용산 → 4호선_이촌 → 4호선_동작 → 4호선_이수 → 4호선_사당 → 2호선_사당 → 2호선_방배 → 2호선_서초 → 2호선_교대 → 2호선_강남
3위 (score_norm_inv=0.934): 1호선_종로3가 → 3호선_종로3가 → 3호선_을지로3가 → 3호선_충무로 → 3호선_동대입구 → 3호선_약수 → 3호선_금호 → 3호선_옥수 → 3호선_압구정 → 3호선_신사 → 3호선_잠원 → 3호선_고속터미널 → 3호선_교대 → 2호선_교대 → 2호선_강남
4위 (score_norm_inv=0.874): 1호선_종로3가 → 1호선_종로5가 → 1호선_동대문 → 1호선_동묘앞 → 1호선_신설동 → 2호선_신설동 → 2호선_용두 → 2호선_신답 → 2호선_용답 → 2호선_성수 → 2호선_건대입구 → 2호선_구의 → 2호선_강변 → 2호선_잠실나루 → 2호선_잠실 → 2호선_잠실새내 → 2호선_종합운동장 → 2호선_삼성 → 2호선_선릉 → 2호선_역삼 → 2호선_강남
5위 (score_norm_inv=0.809): 1호선_종로3가 → 1호선_종로5가 → 1호선_동대문 → 1호선_동묘앞 → 6호선_동묘앞 → 6호선_신당 → 6호선_청구 → 6호선_약수 → 3호선_약수 → 3호선_금호 → 3호선_옥수 → 3호선_압구정 → 3호선_신사 → 3호선_잠원 → 3호선_고속터미널 → 3호선_교대 → 2호선_교대 → 2호선_강남


조건에 따른 필터링

In [ ]:
with open("metro_inside.json", encoding="utf-8") as f:
    inside = json.load(f)

# station_info[(line, station_name)] = 편의시설 dict
station_info = {}
for rec in inside:
    # station_name 필드에 "(호선)" 이 붙어 있을 수 있으니 제거
    base_name = rec["station_name"].split("(")[0]
    for ln in rec["lines_by_name"].split(","):
        station_info[(ln, base_name)] = rec

In [ ]:
# ——————————————————————————————————————————————————————————————————————
# 1) 필수 편의시설 필터링 함수 확장
# ——————————————————————————————————————————————————————————————————————
def filter_routes_by_facilities(
    candidate_routes: list[dict],
    require_lift: bool = False,
    require_elevator: bool = False,
    require_toilet_gate: bool = False,
    require_toilet_cctv: bool = False,
    min_remodel_year: int = None
) -> list[dict]:
    """
    candidate_routes: 평가 전 경로 리스트
    require_lift      : 경로상의 모든 역에 리프트 있어야 함
    require_elevator  : 경로상의 모든 역에 엘리베이터 있어야 함
    require_toilet_gate: 모든 역에 게이트 내부 화장실 있어야 함
    require_toilet_cctv: 모든 역 화장실 입구에 CCTV 있어야 함
    min_remodel_year  : 화장실 리모델 연도가 이 년도 이상이어야 함
    """
    filtered = []
    for r in candidate_routes:
        ok = True
        for node in r["path"]:
            line, station = node.split("_", 1)
            info = station_info.get((line, station), {})

            if require_lift and info.get("total_lift_count", 0) == 0:
                ok = False; break
            if require_elevator and info.get("total_elevator_count", 0) == 0:
                ok = False; break

            if require_toilet_gate:
                # 화장실이 “게이트 내부” 혹은 “내부/외부” 여야 함
                if info.get("toilet_gate_access") not in ("내부", "내부/외부"):
                    ok = False; break

            if require_toilet_cctv:
                # 화장실 입구 CCTV 설치 여부가 'Y' 여야 함
                if info.get("toilet_cctv_at_entrancefh") != "Y":
                    ok = False; break

            if min_remodel_year is not None:
                year_str = info.get("toilet_remodel_yearfh", "")
                try:
                    year = int(year_str[:4])
                except:
                    ok = False; break
                if year < min_remodel_year:
                    ok = False; break

        if ok:
            filtered.append(r)
    return filtered


In [ ]:
def evaluate_weighted_for_candidates(
    metro_graph: nx.Graph,
    candidate_routes: list[dict],
    weights: dict,
    k: int = 10
) -> list[dict]:
    """
    5개 지표(raw) → 합계 기반 비율 정규화(by-sum) →
    가중합 → inverted min–max 정규화 → 상위 k개 반환
    """
    evals = []

    # 1) 원시(raw) 지표 계산
    for r in candidate_routes:
        path = r['path']

        # 1-1) 소요시간
        t = r['total_time_min']

        # 1-2) 정시성: 출발/환승역 99% + 나머지 1%
        grp1 = [path[0]] + [
            v for u,v in zip(path, path[1:])
            if metro_graph[u][v].get('mode') == 'transfer'
        ]
        grp1 = list(dict.fromkeys(grp1))
        grp2 = [n for n in path if n not in grp1]
        def _avg_punct(nodes):
            return (
                sum(get_station_punctuality(n, metro_graph) for n in nodes)
                / len(nodes)
            ) if nodes else 0.0
        punct = 0.99 * _avg_punct(grp1) + 0.01 * _avg_punct(grp2)

        # 1-3) 평균 혼잡도
        cong = sum(
            get_station_congestion(
                metro_graph.nodes[n]['line'],
                metro_graph.nodes[n]['station']
            )
            for n in path
        ) / len(path)

        # 1-4) 쾌적성 합산
        ples = sum(get_station_pleasantness(n) for n in path)

        # 1-5) 환승편의성: 도보+수직이동+환승보행
        walk = r['walk_time_min']
        ln0, st0 = metro_graph.nodes[path[0]]['line'], metro_graph.nodes[path[0]]['station']
        ln1, st1 = metro_graph.nodes[path[-1]]['line'], metro_graph.nodes[path[-1]]['station']
        t_down = vertical_time(ln0, st0)
        t_up   = vertical_time(ln1, st1)
        t_tr   = 0.0
        for u,v in zip(path, path[1:]):
            if metro_graph[u][v].get('mode') == 'transfer':
                ln, st = metro_graph.nodes[u]['line'], metro_graph.nodes[u]['station']
                t_tr += transfer_time_map.get((ln, metro_graph.nodes[v]['line'], st), 0.0)
        trans = walk + t_down + t_tr + t_up

        evals.append({
            **r,
            'time_m':  t,
            'punct_m': punct,
            'cong_m':  cong,
            'ples_m':  ples,
            'trans_m': trans,
        })

    # 2) raw 지표 합계 계산
    sums = {
        'time':  sum(e['time_m']  for e in evals),
        'punct': sum(e['punct_m'] for e in evals),
        'cong':  sum(e['cong_m']  for e in evals),
        'ples':  sum(e['ples_m']  for e in evals),
        'trans': sum(e['trans_m'] for e in evals),
    }

    # 3) by-sum 정규화 + weighted_score 계산
    for e in evals:
        e['time_n']  = e['time_m']  / sums['time']  if sums['time']  > 0 else 0.0
        e['punct_n'] = e['punct_m'] / sums['punct'] if sums['punct'] > 0 else 0.0
        e['cong_n']  = e['cong_m']  / sums['cong']  if sums['cong']  > 0 else 0.0
        e['ples_n']  = e['ples_m']  / sums['ples']  if sums['ples']  > 0 else 0.0
        e['trans_n'] = e['trans_m'] / sums['trans'] if sums['trans'] > 0 else 0.0

        e['weighted_score'] = (
            weights['time']        * e['time_n']    +
            weights['punctuality'] * e['punct_n']   +
            weights['congestion']  * e['cong_n']    +
            weights['pleasant']    * e['ples_n']    +
            weights['transfer']    * e['trans_n']
        )

    if not evals:
        return []

    # 4) weighted_score 오름차순 정렬
    evals.sort(key=lambda x: x['weighted_score'])

    # 5) inverted min–max 정규화: 최적→1, 최악→0
    scores = [e['weighted_score'] for e in evals]
    min_s, max_s = min(scores), max(scores)
    for e in evals:
        if max_s > min_s:
            norm = (e['weighted_score'] - min_s) / (max_s - min_s)
            e['score_norm_inv'] = 1.0 - norm
        else:
            e['score_norm_inv'] = 1.0

    # 6) 상위 k개 반환
    return evals[:k]


In [ ]:
# ——————————————————————————————————————————————————————————————————————
# 2) 사용자 입력 받기
# ——————————————————————————————————————————————————————————————————————
ans_lift  = input("🛗 리프트 필수 경로를 원하시면 Y, 아니면 N: ")
ans_elev  = input("🛗 엘리베이터 필수 경로를 원하시면 Y, 아니면 N: ")
ans_gate  = input("🚻 게이트 내부 화장실이 있는 역만 원하시면 Y, 아니면 N: ")
ans_cctv  = input("📹 화장실 입구 CCTV가 있는 역만 원하시면 Y, 아니면 N: ")
ans_year  = input("🛠 화장실 리모델 연도 최소 기준을 숫자로 입력(예:2018), 아니면 Enter: ")

require_lift         = ans_lift.strip().lower().startswith('y')
require_elevator     = ans_elev.strip().lower().startswith('y')
require_toilet_gate  = ans_gate.strip().lower().startswith('y')
require_toilet_cctv  = ans_cctv.strip().lower().startswith('y')
min_remodel_year     = int(ans_year) if ans_year.isdigit() else None

# ——————————————————————————————————————————————————————————————————————
# 3) 후보 경로 생성 (공통)
# ——————————————————————————————————————————————————————————————————————
routes = get_fastest_subway_routes(
    metro_graph,
    start_nodes_with_walk,
    end_nodes_with_walk,
    k=200
)
fastest    = routes[0]['total_time_min']
candidates = [r for r in routes if r['total_time_min'] <= 2 * fastest]

# ——————————————————————————————————————————————————————————————————————
# 4) 필터링 적용
# ——————————————————————————————————————————————————————————————————————
candidates = filter_routes_by_facilities(
    candidates,
    require_lift=require_lift,
    require_elevator=require_elevator,
    require_toilet_gate=require_toilet_gate,
    require_toilet_cctv=require_toilet_cctv,
    min_remodel_year=min_remodel_year
)
print(f"필터 후 후보 경로 수: {len(candidates)}")

🛗 리프트 필수 경로를 원하시면 Y, 아니면 N: N
🛗 엘리베이터 필수 경로를 원하시면 Y, 아니면 N: Y
🚻 게이트 내부 화장실이 있는 역만 원하시면 Y, 아니면 N: Y
📹 화장실 입구 CCTV가 있는 역만 원하시면 Y, 아니면 N: Y
🛠 화장실 리모델 연도 최소 기준을 숫자로 입력(예:2018), 아니면 Enter: 2018
필터 후 후보 경로 수: 0


In [ ]:
# ——————————————————————————————————————————————————————————————————————
# 5) 우선순위별 평가 및 출력 (기존 로직)
# ——————————————————————————————————————————————————————————————————————
for label, weights in priority_weights.items():
    print(f"\n=== {label} Top 5 경로 ===")
    top5 = evaluate_weighted_for_candidates(
        metro_graph,
        candidates,
        weights,
        k=5
    )
    if not top5:
        print("  (조건에 맞는 경로가 없습니다.)")
        continue
    for i, r in enumerate(top5, 1):
        print(f"{i}위 (score_norm_inv={r['score_norm_inv']:.3f}): {' → '.join(r['path'])}")


=== 정시성 우선 Top 5 경로 ===
  (조건에 맞는 경로가 없습니다.)

=== 소요시간 우선 Top 5 경로 ===
  (조건에 맞는 경로가 없습니다.)

=== 환승편의 우선 Top 5 경로 ===
  (조건에 맞는 경로가 없습니다.)

=== 혼잡도 우선 Top 5 경로 ===
  (조건에 맞는 경로가 없습니다.)

=== 쾌적성 우선 Top 5 경로 ===
  (조건에 맞는 경로가 없습니다.)
